In [3]:
# Seeds and Sources

# Trazer e analisar o arquivo seed_full_moon_date.csv
# Todos os dias que houve Lua Cheia nos ultimos 10 anos

In [5]:
# Comando

# dbt seed 
# Ira popular os dados no Snowflake

In [6]:
# Criar arquivo models/mart/mart_fullmoon_reviews.sql
# Adicionar query para analisar dados de full moon with reviews

# dbt run

In [7]:
# Sources sao uma abrstracao sobre o input data

In [8]:
# Para criar o arquivo models/sources.yml

a = """
version: 2

sources:
  - name: airbnb
    schema: raw
    tables:
      - name: listings
        identifier: raw_listings

      - name: hosts
        identifier: raw_hosts
      - name: reviews
        identifier: raw_reviews

"""

In [1]:
# Alterar a referencia do snowflake para a source do dbt que foi criado
# src/srs_hosts.sql
# src/src_listings.sql
# src/src_reviews.sql

"""
with raw_listings as (select * from {{ source("airbnb", "listings") }})
"""

"""
with raw_hosts as (select * from {{ source("airbnb", "hosts") }})
"""

"""
with raw_reviews as (select * from {{ source("airbnb", "reviews") }})
"""

'\nwith raw_reviews as (select * from {{ source("airbnb", "reviews") }})\n'

In [2]:
# Rodar comando

# dbt compile

# apenas para verificar se os modelos que criamos fazem sentido

In [12]:
# Source Freshness

# arquivo sources.yaml

"""
      - name: hosts
        identifier: raw_hosts
      - name: reviews
        identifier: raw_reviews
        loaded_at_field: date
        freshness:
          warn_after: {count: 1, period: hour}
           -- if find any data with less than 1 hour fresh will wanr
          error_after: {count: 24, period: hour}
          -- if find 24 or more rows less than 1 hour fresh will return error
"""

'\n      - name: hosts\n        identifier: raw_hosts\n      - name: reviews\n        identifier: raw_reviews\n        loaded_at_field: date\n        freshness:\n          warn_after: {count: 1, period: hour}\n          error_after: {count: 24, period: hour}\n'

In [3]:
# rodar comando para ver o dado que adicionamos agora com WARN

# dbt source freshness
"""
19:54:59  Running with dbt=1.7.14
19:55:00  Registered adapter: snowflake=1.7.1
19:55:00  Found 1 seed, 8 models, 3 sources, 0 exposures, 0 metrics, 430 macros, 0 groups, 0 semantic models
19:55:00  
19:55:01  Concurrency: 1 threads (target='dev')
19:55:01  
19:55:01  1 of 1 START freshness of airbnb.reviews ....................................... [RUN]
19:55:02  1 of 1 WARN freshness of airbnb.reviews ........................................ [WARN in 0.72s]
19:55:02  Done.
"""

"\n19:54:59  Running with dbt=1.7.14\n19:55:00  Registered adapter: snowflake=1.7.1\n19:55:00  Found 1 seed, 8 models, 3 sources, 0 exposures, 0 metrics, 430 macros, 0 groups, 0 semantic models\n19:55:00  \n19:55:01  Concurrency: 1 threads (target='dev')\n19:55:01  \n19:55:01  1 of 1 START freshness of airbnb.reviews ....................................... [RUN]\n19:55:02  1 of 1 WARN freshness of airbnb.reviews ........................................ [WARN in 0.72s]\n19:55:02  Done.\n"

In [14]:
# Snapshots

# Iremos fazer um snapshot da camada raw (do input data) assim que ele tiver 
# qualquer mudanca iremos capturar um snapshot

In [15]:
# Criar arquivo snapshots/scd_raw_listings.sql

In [4]:
"""
{% snapshot scd_raw_listings %}
    {{
        config(
            target_schema="DEV",
            unique_key="id",
            strategy="timestamp",
            updated_at="updated_at",
            invalidate_hard_deletes=True,
        )
    }}
    select *
    from {{ source("airbnb", "listings") }}
{% endsnapshot %}

"""

'\n{% snapshot scd_raw_listings %}\n    {{\n        config(\n            target_schema="DEV",\n            unique_key="id",\n            strategy="timestamp",\n            updated_at="updated_at",\n            invalidate_hard_deletes=True,\n        )\n    }}\n    select *\n    from {{ source("airbnb", "listings") }}\n{% endsnapshot %}\n\n'

In [16]:
# Rodar Comando

# dbt snapshot

In [17]:
# Verificar nova tabela criada no SF -> 
# scd_raw_listings.sql

In [ ]:
# Dentro do snowfalke rodar o seguinte comando para testar o snapshot

"""
UPDATE AIRBNB.RAW.RAW_LISTINGS SET MINIMUM_NIGHTS=30, updated_at=CURRENT_TIMESTAMP() WHERE ID=3176;
"""

# dbt snapshot

"""
SELECT * FROM AIRBNB.DEV.SCD_RAW_LISTINGS WHERE ID=3176;
"""